In [1]:
import datetime as dt
from pathlib import Path
import os

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 데이터 불러오기

In [2]:
data_path = Path(os.getenv('HOME')+'/aiffel/exp12/yoochoose/data/') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.

# 데이터 확인

In [3]:
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


In [4]:
data['UserId'].nunique(), data['ItemId'].nunique()

(6040, 3706)

# 유저 아이디 수를 구해 통계 도출

In [5]:
session_length = data.groupby('UserId').size()
session_length

UserId
1        53
2       129
3        51
4        21
5       198
       ... 
6036    888
6037    202
6038     20
6039    123
6040    341
Length: 6040, dtype: int64

In [6]:
session_length.describe()

count    6040.000000
mean      165.597517
std       192.747029
min        20.000000
25%        44.000000
50%        96.000000
75%       208.000000
max      2314.000000
dtype: float64

# 이상치 판단 기준에 맞춰 바운더리를 설정하고 이를 넘은 데이터를 삭제

In [7]:
q3 = session_length.quantile(0.75)
q1 = session_length.quantile(0.25)
upper_boundary = q3+3/2*(q3-q1)
upper_boundary

454.0

In [8]:
drop_index = session_length[session_length>upper_boundary].index
data.drop(index=drop_index, inplace=True)

# 최종 날짜로부터 1년 전 데이터만 남긴다

In [9]:
oldest, latest = data['Time'].min(), data['Time'].max()
print(oldest)
print(latest)

956703932
1046454590


In [10]:
year_ago = latest - (365*24*60*60)     # 최종 날짜로부터 1년 이전 날짜를 구한다.  
data = data[data['Time'] > year_ago]   # 방금 구한 날짜 이후의 데이터만 모은다. 

In [11]:
data

,UserId,ItemId,Rating,Time
5170,36,1387,5,1015904819
5267,36,1201,4,1015904819
5122,36,1291,5,1015904836
5123,36,2167,5,1015904905
5290,36,2951,4,1015904905
...,...,...,...,...
992358,5996,3835,3,1020113184
992279,5996,2422,3,1020113225
992702,5996,168,3,1031058746
992459,5996,339,4,1033997079


# 세션을 구하기 위해 각 아이디별 사용시간 기록 사이의 차이 diff를 구한다 

In [12]:
data = data.sort_values(['UserId', 'Time'])
data['diff'] = data.groupby('UserId')['Time'].diff()

In [13]:
data.head(50)

,UserId,ItemId,Rating,Time,diff
5170,36,1387,5,1015904819,NaN
5267,36,1201,4,1015904819,0.0
5122,36,1291,5,1015904836,17.0
5123,36,2167,5,1015904905,69.0
5290,36,2951,4,1015904905,0.0
5073,36,1912,5,1015904924,19.0
5366,36,2985,4,1015904941,17.0
5305,36,2527,3,1015904957,16.0
5145,36,198,5,1015904988,31.0
5296,36,1147,4,1015905111,123.0


# 각 아이디의 첫번째 열은 첫 데이터라 차이가 없으므로 0으로 기록

In [14]:
data = data.fillna(0)

In [15]:
data.head(50)

,UserId,ItemId,Rating,Time,diff
5170,36,1387,5,1015904819,0.0
5267,36,1201,4,1015904819,0.0
5122,36,1291,5,1015904836,17.0
5123,36,2167,5,1015904905,69.0
5290,36,2951,4,1015904905,0.0
5073,36,1912,5,1015904924,19.0
5366,36,2985,4,1015904941,17.0
5305,36,2527,3,1015904957,16.0
5145,36,198,5,1015904988,31.0
5296,36,1147,4,1015905111,123.0


In [16]:
data.tail(50)

,UserId,ItemId,Rating,Time,diff
992550,5996,2247,4,1020110044,0.0
992783,5996,552,4,1020110044,0.0
992575,5996,3253,3,1020110139,95.0
992349,5996,3809,3,1020110188,49.0
992554,5996,783,3,1020110188,0.0
992602,5996,2672,5,1020110214,26.0
992303,5996,2470,3,1020110294,80.0
992305,5996,203,3,1020110294,0.0
992562,5996,799,3,1020110294,0.0
992203,5996,3791,3,1020110319,25.0


# diff에 대한 통계 확인

In [17]:
data.describe()

,UserId,ItemId,Rating,Time,diff
count,21701.000000,21701.000000,21701.000000,2.170100e+04,2.170100e+04
mean,2754.583061,1982.844431,3.446523,1.029144e+09,2.723040e+05
std,1752.600720,1135.279101,1.106853,9.362673e+06,1.507050e+06
min,36.000000,1.000000,1.000000,1.014923e+09,0.000000e+00
25%,1194.000000,1090.000000,3.000000,1.020427e+09,0.000000e+00
50%,2594.000000,2000.000000,4.000000,1.028393e+09,2.800000e+01
75%,4305.000000,2959.000000,4.000000,1.037423e+09,1.180000e+02
max,5996.000000,3952.000000,5.000000,1.046455e+09,3.076749e+07


# diff에 대해 이상치 기준선을 계산한다

In [18]:
diff_upper_boundary = 3/2*data['diff'].quantile(0.75)
diff_upper_boundary

177.0

In [19]:
data = data.reset_index()

# 사용시간 차이가 이상치 기준을 넘어서면 연속된 세션이 아니라고 판단하고 이를 세션 분리 기준으로 하여 세션을 구분한다.

In [20]:
timediff_idx = data.index[data['diff']>diff_upper_boundary] # 이상치 기준으로 인덱스 추출
UserId_idx = data.index[data['UserId'] != data['UserId'].shift(1)] # 이전 행과 UserId가 달라진 인덱스 추출

total_idx = list(timediff_idx.append(UserId_idx).unique()) # 위에서 구한 두인덱스를 합한 후 유니크한 인덱스만 추출
all_idx = list(data.index) # data의 전체 인덱스

value = 0
dic = {}
data['SessionId'] = 0

# 전체 인덱 스 중 total_idx가 포함되어 있으면 +1 후 그 값을 대입하고 아니면 이전값 그대로 대입한다.
for i in all_idx:
    if i in total_idx:
        data.loc[i, 'SessionId'] = value
        value += 1
    else:
        data.loc[i, 'SessionId'] = value

In [21]:
data.head(20)

,index,UserId,ItemId,Rating,Time,diff,SessionId
0,5170,36,1387,5,1015904819,0.0,0
1,5267,36,1201,4,1015904819,0.0,1
2,5122,36,1291,5,1015904836,17.0,1
3,5123,36,2167,5,1015904905,69.0,1
4,5290,36,2951,4,1015904905,0.0,1
5,5073,36,1912,5,1015904924,19.0,1
6,5366,36,2985,4,1015904941,17.0,1
7,5305,36,2527,3,1015904957,16.0,1
8,5145,36,198,5,1015904988,31.0,1
9,5296,36,1147,4,1015905111,123.0,1


In [22]:
data.tail(20)

,index,UserId,ItemId,Rating,Time,diff,SessionId
21681,992214,5996,5,4,1020111183,42.0,5077
21682,992523,5996,719,4,1020111183,0.0,5077
21683,992579,5996,2605,5,1020111183,0.0,5077
21684,992620,5996,466,3,1020111221,38.0,5077
21685,992677,5996,2375,3,1020111221,0.0,5077
21686,992583,5996,2616,3,1020111375,154.0,5077
21687,992724,5996,3707,5,1020111375,0.0,5077
21688,992553,5996,781,5,1020111524,149.0,5077
21689,992775,5996,542,3,1020111524,0.0,5077
21690,992689,5996,2701,3,1020111976,452.0,5077


# 한칸씩 밀린 세션 값을 조정한다

In [23]:
data['SessionId'] = data['SessionId'].shift(periods=-1)

In [24]:
data.tail()

,index,UserId,ItemId,Rating,Time,diff,SessionId
21696,992358,5996,3835,3,1020113184,792.0,5079.0
21697,992279,5996,2422,3,1020113225,41.0,5079.0
21698,992702,5996,168,3,1031058746,10945521.0,5080.0
21699,992459,5996,339,4,1033997079,2938333.0,5081.0
21700,992387,5996,3882,3,1036428955,2431876.0,NaN


In [25]:
data['SessionId'] = data['SessionId'].fillna(5082)
data['SessionId'] = data['SessionId'].astype('int')

In [26]:
data.tail()

,index,UserId,ItemId,Rating,Time,diff,SessionId
21696,992358,5996,3835,3,1020113184,792.0,5079
21697,992279,5996,2422,3,1020113225,41.0,5079
21698,992702,5996,168,3,1031058746,10945521.0,5080
21699,992459,5996,339,4,1033997079,2938333.0,5081
21700,992387,5996,3882,3,1036428955,2431876.0,5082


In [27]:
data.describe()

,index,UserId,ItemId,Rating,Time,diff,SessionId
count,21701.000000,21701.000000,21701.000000,21701.000000,2.170100e+04,2.170100e+04,21701.000000
mean,454849.672089,2754.583061,1982.844431,3.446523,1.029144e+09,2.723040e+05,2685.439888
std,292783.017189,1752.600720,1135.279101,1.106853,9.362673e+06,1.507050e+06,1451.100806
min,5025.000000,36.000000,1.000000,1.000000,1.014923e+09,0.000000e+00,1.000000
25%,193901.000000,1194.000000,1090.000000,3.000000,1.020427e+09,0.000000e+00,1369.000000
50%,427329.000000,2594.000000,2000.000000,4.000000,1.028393e+09,2.800000e+01,2912.000000
75%,718964.000000,4305.000000,2959.000000,4.000000,1.037423e+09,1.180000e+02,3920.000000
max,992787.000000,5996.000000,3952.000000,5.000000,1.046455e+09,3.076749e+07,5082.000000


# 세션의 길이가 1이거나 클릭수가 적은 작품을 삭제한다

In [28]:
# short_session을 제거한 다음 unpopular item을 제거하면 다시 길이가 1인 session이 생길 수 있습니다.
# 이를 위해 반복문을 통해 지속적으로 제거 합니다.
def cleanse_recursive(data: pd.DataFrame, shortest, least_click) -> pd.DataFrame:
    while True:
        before_len = len(data)
        data = cleanse_short_session(data, shortest)
        data = cleanse_unpopular_item(data, least_click)
        after_len = len(data)
        if before_len == after_len:
            break
    return data


def cleanse_short_session(data: pd.DataFrame, shortest):
    session_len = data.groupby('SessionId').size()
    session_use = session_len[session_len >= shortest].index
    data = data[data['SessionId'].isin(session_use)]
    return data


def cleanse_unpopular_item(data: pd.DataFrame, least_click):
    item_popular = data.groupby('ItemId').size()
    item_use = item_popular[item_popular >= least_click].index
    data = data[data['ItemId'].isin(item_use)]
    return data

In [29]:
data = cleanse_recursive(data, shortest=2, least_click=5)

In [30]:
data

,index,UserId,ItemId,Rating,Time,diff,SessionId
0,5170,36,1387,5,1015904819,0.0,1
1,5267,36,1201,4,1015904819,0.0,1
2,5122,36,1291,5,1015904836,17.0,1
3,5123,36,2167,5,1015904905,69.0,1
4,5290,36,2951,4,1015904905,0.0,1
...,...,...,...,...,...,...,...
21688,992553,5996,781,5,1020111524,149.0,5077
21689,992775,5996,542,3,1020111524,0.0,5077
21690,992689,5996,2701,3,1020111976,452.0,5078
21691,992304,5996,2471,2,1020112109,133.0,5078


In [31]:
data.drop('diff', axis=1, inplace=True)

In [32]:
data

,index,UserId,ItemId,Rating,Time,SessionId
0,5170,36,1387,5,1015904819,1
1,5267,36,1201,4,1015904819,1
2,5122,36,1291,5,1015904836,1
3,5123,36,2167,5,1015904905,1
4,5290,36,2951,4,1015904905,1
...,...,...,...,...,...,...
21688,992553,5996,781,5,1020111524,5077
21689,992775,5996,542,3,1020111524,5077
21690,992689,5996,2701,3,1020111976,5078
21691,992304,5996,2471,2,1020112109,5078


# 최근 1달 데이터를 테스트 데이터, 그 뒤로 2달간의 데이터를 검정용 데이터로 분리하고 나머지를 학습용 데이터로 분류한다
# 1년 중 테스트 데이터로 사용할 최근 1달을 뺀 11달에서 충분히 검정용 데이터를 사용하기 위해 검정용 데이터를 2달치로 설정했습니다

In [33]:
data['Time'].max(), data['Time'].min()

(1046454590, 1014923199)

In [34]:
test_time = data['Time'].max() - 30*24*60*60

In [35]:
test_time

1043862590

In [36]:
valid_time = test_time - 61*24*60*60

In [37]:
valid_time

1038592190

In [38]:
train_df = data[data['Time'] <= valid_time]
train_df.tail()

,index,UserId,ItemId,Rating,Time,SessionId
21688,992553,5996,781,5,1020111524,5077
21689,992775,5996,542,3,1020111524,5077
21690,992689,5996,2701,3,1020111976,5078
21691,992304,5996,2471,2,1020112109,5078
21695,992466,5996,3564,3,1020112392,5078


In [39]:
test_df = data[data['Time'] > test_time]
test_df

,index,UserId,ItemId,Rating,Time,SessionId
804,28281,195,410,2,1044987808,206
805,28389,195,2791,3,1044987808,206
806,28391,195,2792,2,1044987808,206
813,27846,195,1356,2,1045358834,212
814,27901,195,1374,3,1045358834,212
...,...,...,...,...,...,...
21553,984731,5950,3948,4,1046369637,5061
21554,984682,5950,3578,4,1046369670,5061
21555,984475,5950,3793,3,1046369710,5061
21556,984660,5950,3555,2,1046369737,5061


In [40]:
temp_df = data[data['Time'] <= test_time]
val_df = temp_df[temp_df['Time'] > valid_time]
val_df

,index,UserId,ItemId,Rating,Time,SessionId
19,5200,36,2269,5,1040544350,6
20,5184,36,2694,3,1040544494,6
21,5364,36,3786,4,1040544521,6
22,5262,36,2369,4,1040544564,6
23,5069,36,2713,1,1040544662,6
...,...,...,...,...,...,...
21505,983989,5948,224,4,1043068072,5048
21506,984167,5948,1283,5,1043068249,5048
21507,984206,5948,1304,5,1043068456,5049
21508,984173,5948,1288,4,1043068509,5049


In [41]:
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Sessions: {data["SessionId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["Time"].min()}\n'
          f'\t Last Time : {data["Time"].max()}\n')

# 분리된 데이터들의 통계

In [42]:
stats_info(train_df, 'train')
stats_info(val_df, 'valid')
stats_info(test_df, 'test')

* train Set Stats Info
	 Events: 11900
	 Sessions: 1433
	 Items: 1326
	 First Time : 1014923199
	 Last Time : 1038587440

* valid Set Stats Info
	 Events: 2237
	 Sessions: 293
	 Items: 998
	 First Time : 1038608526
	 Last Time : 1043823522

* test Set Stats Info
	 Events: 1148
	 Sessions: 118
	 Items: 727
	 First Time : 1043869016
	 Last Time : 1046454590



# 데이터 셋 생성

In [43]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(train_df['ItemId'].unique())}
def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

train_df = indexing(train_df, id2idx)
val_df = indexing(val_df, id2idx)
test_df = indexing(test_df, id2idx)

In [44]:
data_path = Path('data')
save_path = data_path / 'processed'
save_path.mkdir(parents=True, exist_ok=True)

train_df.to_pickle(save_path / 'train.pkl')
val_df.to_pickle(save_path / 'valid.pkl')
test_df.to_pickle(save_path / 'test.pkl')

In [45]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['SessionId'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['SessionId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('SessionId').size().cumsum()
        return offsets

In [46]:
tr_dataset = SessionDataset(train_df)
tr_dataset.df.head(10)

,index,UserId,ItemId,Rating,Time,SessionId,item_idx
0,5170,36,1387,5,1015904819,1,0
1,5267,36,1201,4,1015904819,1,1
2,5122,36,1291,5,1015904836,1,2
3,5123,36,2167,5,1015904905,1,3
4,5290,36,2951,4,1015904905,1,4
5,5073,36,1912,5,1015904924,1,5
6,5366,36,2985,4,1015904941,1,6
8,5145,36,198,5,1015904988,1,7
9,5296,36,1147,4,1015905111,1,8
10,5034,36,581,4,1015905127,1,9


In [47]:
tr_dataset.click_offsets

array([    0,    11,    13, ..., 11852, 11897, 11900], dtype=int32)

In [48]:
tr_dataset.session_idx

array([   0,    1,    2, ..., 1430, 1431, 1432])

In [49]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):  
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.
        
        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]  
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i  
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished

In [50]:
tr_data_loader = SessionDataLoader(tr_dataset, batch_size=4)
tr_dataset.df.head(15)

,index,UserId,ItemId,Rating,Time,SessionId,item_idx
0,5170,36,1387,5,1015904819,1,0
1,5267,36,1201,4,1015904819,1,1
2,5122,36,1291,5,1015904836,1,2
3,5123,36,2167,5,1015904905,1,3
4,5290,36,2951,4,1015904905,1,4
5,5073,36,1912,5,1015904924,1,5
6,5366,36,2985,4,1015904941,1,6
8,5145,36,198,5,1015904988,1,7
9,5296,36,1147,4,1015905111,1,8
10,5034,36,581,4,1015905127,1,9


In [51]:
tr_data_loader = SessionDataLoader(tr_dataset, batch_size=4)
tr_dataset.df.head(15)

,index,UserId,ItemId,Rating,Time,SessionId,item_idx
0,5170,36,1387,5,1015904819,1,0
1,5267,36,1201,4,1015904819,1,1
2,5122,36,1291,5,1015904836,1,2
3,5123,36,2167,5,1015904905,1,3
4,5290,36,2951,4,1015904905,1,4
5,5073,36,1912,5,1015904924,1,5
6,5366,36,2985,4,1015904941,1,6
8,5145,36,198,5,1015904988,1,7
9,5296,36,1147,4,1015905111,1,8
10,5034,36,581,4,1015905127,1,9


In [52]:
iter_ex = iter(tr_data_loader)

In [53]:
inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [ 0 11 13 16]
Label Item Idx are :       [ 1 12 14 17]
Previous Masked Input Idx are []


# MRR과 Recall@K 정의

In [54]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

# 모델링 준비

In [55]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [56]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [57]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['SessionId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

In [58]:
# train 셋으로 학습하면서 valid 셋으로 검증합니다.
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr['SessionId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  # valid set에 대해 검증합니다.

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다. 
                                             # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data['SessionId'].nunique()
    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(pred, direction='DESCENDING')  # softmax 값이 큰 순서대로 sorting 합니다.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

In [59]:
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

# K의 통계적 도출을 위해 SessionId에 대한 통계 추출

In [60]:
s_k = data['SessionId'].value_counts()

In [61]:
s_k.describe()

count    1844.000000
mean        8.289046
std        16.427432
min         2.000000
25%         2.000000
50%         3.000000
75%         7.000000
max       220.000000
Name: SessionId, dtype: float64

# 세션 아이디의 이상치 기준을 K의 기준점으로 활용

In [62]:
k = int(s_k.quantile(0.75) + 3/2*(s_k.quantile(0.75) - s_k.quantile(0.25)))
print(k)

14


# 모델링 1
### batch size = 64
### drop rate = 0.1
### learning rate = 0.001
### epochs = 10
### k = 14

In [63]:
args1 = Args(train_df, val_df, test_df, batch_size=64, hsz=50, drop_rate=0.1, lr=0.001, epochs=10, k=k)

In [64]:
model1 = create_model(args1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(64, 1, 1326)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      206700    
_________________________________________________________________
dropout (Dropout)            (64, 50)                  0         
_________________________________________________________________
dense (Dense)                (64, 1326)                67626     
Total params: 274,326
Trainable params: 274,326
Non-trainable params: 0
_________________________________________________________________


In [65]:
train_model(model1, args1)

Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=7.1]      

	 - Recall@14 epoch 1: 0.025735
	 - MRR@14    epoch 1: 0.005785



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.0156, train_loss=7.07]

	 - Recall@14 epoch 2: 0.025735
	 - MRR@14    epoch 2: 0.005723



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=7.03]     

...

	 - Recall@14 epoch 8: 0.037684
	 - MRR@14    epoch 8: 0.012085



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=6.57]     

	 - Recall@14 epoch 9: 0.040441
	 - MRR@14    epoch 9: 0.013208



Evaluation:  57%|█████▋    | 17/30 [00:05<00:04,  3.06it/s]

	 - Recall@14 epoch 10: 0.042279
	 - MRR@14    epoch 10: 0.015043



In [66]:
test_model(model1, args1, test_df)

Evaluation:  25%|██▌       | 4/16 [00:01<00:05,  2.34it/s]

	 - Recall@14: 0.039062
	 - MRR@14: 0.005715



# 모델링 2
### batch size = 64
### drop rate = 0.2
### learning rate = 0.001
### epochs = 10
### k = 14

In [67]:
args2 = Args(train_df, val_df, test_df, batch_size=64, hsz=50, drop_rate=0.2, lr=0.001, epochs=10, k=k)

In [68]:
model2 = create_model(args2)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(64, 1, 1326)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      206700    
_________________________________________________________________
dropout_1 (Dropout)          (64, 50)                  0         
_________________________________________________________________
dense_1 (Dense)              (64, 1326)                67626     
Total params: 274,326
Trainable params: 274,326
Non-trainable params: 0
_________________________________________________________________


In [69]:
train_model(model2, args2)

Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.0156, train_loss=7.1] 

	 - Recall@14 epoch 1: 0.026654
	 - MRR@14    epoch 1: 0.005036



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=7.06]     

	 - Recall@14 epoch 2: 0.025735
	 - MRR@14    epoch 2: 0.005644



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=7.02]     

...

	 - Recall@14 epoch 8: 0.035846
	 - MRR@14    epoch 8: 0.013528



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=6.8]      

	 - Recall@14 epoch 9: 0.039522
	 - MRR@14    epoch 9: 0.012393



Evaluation:  57%|█████▋    | 17/30 [00:05<00:04,  3.06it/s]

	 - Recall@14 epoch 10: 0.041360
	 - MRR@14    epoch 10: 0.014730



In [70]:
test_model(model2, args2, test_df)

Evaluation:  25%|██▌       | 4/16 [00:01<00:05,  2.32it/s]

	 - Recall@14: 0.031250
	 - MRR@14: 0.009406



# 모델링 3
### batch size = 64
### drop rate = 0.5
### learning rate = 0.001
### epochs = 10
### k = 14

In [71]:
args3 = Args(train_df, val_df, test_df, batch_size=64, hsz=50, drop_rate=0.5, lr=0.001, epochs=10, k=k)

In [72]:
model3 = create_model(args3)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(64, 1, 1326)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      206700    
_________________________________________________________________
dropout_2 (Dropout)          (64, 50)                  0         
_________________________________________________________________
dense_2 (Dense)              (64, 1326)                67626     
Total params: 274,326
Trainable params: 274,326
Non-trainable params: 0
_________________________________________________________________


In [73]:
train_model(model3, args3)

Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=7.1]      

	 - Recall@14 epoch 1: 0.034926
	 - MRR@14    epoch 1: 0.007339



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=7.06]     

	 - Recall@14 epoch 2: 0.026654
	 - MRR@14    epoch 2: 0.005691



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.0156, train_loss=6.91]

...

	 - Recall@14 epoch 8: 0.034007
	 - MRR@14    epoch 8: 0.010329



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=6.9]      

	 - Recall@14 epoch 9: 0.038603
	 - MRR@14    epoch 9: 0.012343



Evaluation:  57%|█████▋    | 17/30 [00:05<00:04,  3.04it/s]

	 - Recall@14 epoch 10: 0.039522
	 - MRR@14    epoch 10: 0.014580



In [74]:
test_model(model3, args3, test_df)

Evaluation:  25%|██▌       | 4/16 [00:01<00:05,  2.33it/s]

	 - Recall@14: 0.039062
	 - MRR@14: 0.010111



# 모델링 4
### batch size = 64
### drop rate = 0.1
### learning rate = 0.01
### epochs = 10
### k = 14

In [75]:
args4 = Args(train_df, val_df, test_df, batch_size=64, hsz=50, drop_rate=0.1, lr=0.01, epochs=10, k=k)

In [76]:
model4 = create_model(args4)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(64, 1, 1326)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      206700    
_________________________________________________________________
dropout_3 (Dropout)          (64, 50)                  0         
_________________________________________________________________
dense_3 (Dense)              (64, 1326)                67626     
Total params: 274,326
Trainable params: 274,326
Non-trainable params: 0
_________________________________________________________________


In [77]:
train_model(model4, args4)

Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=6.9]      

	 - Recall@14 epoch 1: 0.036765
	 - MRR@14    epoch 1: 0.012613



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.0156, train_loss=6.62]

	 - Recall@14 epoch 2: 0.046875
	 - MRR@14    epoch 2: 0.018082



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.0938, train_loss=5.34]

...

	 - Recall@14 epoch 8: 0.059743
	 - MRR@14    epoch 8: 0.022882



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.516, train_loss=1.67]

	 - Recall@14 epoch 9: 0.060662
	 - MRR@14    epoch 9: 0.023396



Evaluation:  57%|█████▋    | 17/30 [00:05<00:04,  3.10it/s]

	 - Recall@14 epoch 10: 0.063419
	 - MRR@14    epoch 10: 0.023483



In [78]:
test_model(model4, args4, test_df)

Evaluation:  25%|██▌       | 4/16 [00:01<00:05,  2.26it/s]

	 - Recall@14: 0.054688
	 - MRR@14: 0.010108



# 모델링 5
### batch size = 64
### drop rate = 0.1
### learning rate = 0.1
### epochs = 10
### k = 14

In [79]:
args5 = Args(train_df, val_df, test_df, batch_size=64, hsz=50, drop_rate=0.1, lr=0.1, epochs=10, k=k)

In [80]:
model5 = create_model(args5)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(64, 1, 1326)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      206700    
_________________________________________________________________
dropout_4 (Dropout)          (64, 50)                  0         
_________________________________________________________________
dense_4 (Dense)              (64, 1326)                67626     
Total params: 274,326
Trainable params: 274,326
Non-trainable params: 0
_________________________________________________________________


In [81]:
train_model(model5, args5)

Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=7.08]     

	 - Recall@14 epoch 1: 0.042279
	 - MRR@14    epoch 1: 0.011337



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=7.49]     

	 - Recall@14 epoch 2: 0.031250
	 - MRR@14    epoch 2: 0.010941



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.0156, train_loss=7.66]

...

	 - Recall@14 epoch 8: 0.030331
	 - MRR@14    epoch 8: 0.010005



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.0469, train_loss=9.07]

	 - Recall@14 epoch 9: 0.035846
	 - MRR@14    epoch 9: 0.010308



Evaluation:  57%|█████▋    | 17/30 [00:05<00:04,  3.02it/s]

	 - Recall@14 epoch 10: 0.036765
	 - MRR@14    epoch 10: 0.009702



In [82]:
test_model(model5, args5, test_df)

Evaluation:  25%|██▌       | 4/16 [00:01<00:05,  2.23it/s]

	 - Recall@14: 0.027344
	 - MRR@14: 0.005704



***

# drop out과 learning rate를 변경하며 시도해 본 결과 1번 모델에서 drop out을 0.5로 변경했을 때와 learning rate를 0.1로 변경했을 때 가장 결과가 좋아 이를 기준으로 다시 다양한 하이퍼파라미터를 시도

# 모델링 6
### batch size = 32
### drop rate = 0.1
### learning rate = 0.01
### epochs = 10
### k = 14

In [83]:
args6 = Args(train_df, val_df, test_df, batch_size=32, hsz=50, drop_rate=0.1, lr=0.01, epochs=10, k=k)

In [84]:
model6 = create_model(args6)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(32, 1, 1326)]           0         
_________________________________________________________________
GRU (GRU)                    [(32, 50), (32, 50)]      206700    
_________________________________________________________________
dropout_5 (Dropout)          (32, 50)                  0         
_________________________________________________________________
dense_5 (Dense)              (32, 1326)                67626     
Total params: 274,326
Trainable params: 274,326
Non-trainable params: 0
_________________________________________________________________


In [85]:
train_model(model6, args6)

Train:   0%|          | 0/327 [00:00<?, ?it/s, accuracy=0.0312, train_loss=6.72]

	 - Recall@14 epoch 1: 0.049851
	 - MRR@14    epoch 1: 0.018007



Train:   0%|          | 0/327 [00:00<?, ?it/s, accuracy=0, train_loss=6.48]     

	 - Recall@14 epoch 2: 0.052827
	 - MRR@14    epoch 2: 0.021825



Train:   0%|          | 0/327 [00:00<?, ?it/s, accuracy=0.0938, train_loss=5.01]

...

	 - Recall@14 epoch 8: 0.056548
	 - MRR@14    epoch 8: 0.019639



Train:   0%|          | 0/327 [00:00<?, ?it/s, accuracy=0.562, train_loss=1.81]

	 - Recall@14 epoch 9: 0.050595
	 - MRR@14    epoch 9: 0.020797



Evaluation:  70%|███████   | 42/60 [00:07<00:03,  5.42it/s]

	 - Recall@14 epoch 10: 0.052827
	 - MRR@14    epoch 10: 0.019694



In [86]:
test_model(model6, args6, test_df)

Evaluation:  47%|████▋     | 15/32 [00:03<00:04,  3.94it/s]

	 - Recall@14: 0.050000
	 - MRR@14: 0.021337



# 모델링 7
### batch size = 64
### drop rate = 0.2
### learning rate = 0.01
### epochs = 10
### k = 14

In [ ]:
args7 = Args(train_df, val_df, test_df, batch_size=64, hsz=50, drop_rate=0.2, lr=0.01, epochs=10, k=k)

In [88]:
model7 = create_model(args7)

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(64, 1, 1326)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      206700    
_________________________________________________________________
dropout_6 (Dropout)          (64, 50)                  0         
_________________________________________________________________
dense_6 (Dense)              (64, 1326)                67626     
Total params: 274,326
Trainable params: 274,326
Non-trainable params: 0
_________________________________________________________________


In [89]:
train_model(model7, args7)

Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=6.81]     

	 - Recall@14 epoch 1: 0.045037
	 - MRR@14    epoch 1: 0.009706



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.0312, train_loss=6.36]

	 - Recall@14 epoch 2: 0.043199
	 - MRR@14    epoch 2: 0.018797



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.0469, train_loss=5.47]

...

	 - Recall@14 epoch 8: 0.055147
	 - MRR@14    epoch 8: 0.023057



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.375, train_loss=2.37]

	 - Recall@14 epoch 9: 0.052390
	 - MRR@14    epoch 9: 0.020204



Evaluation:  57%|█████▋    | 17/30 [00:05<00:04,  3.06it/s]

	 - Recall@14 epoch 10: 0.049632
	 - MRR@14    epoch 10: 0.022077



In [90]:
test_model(model7, args7, test_df)

Evaluation:  25%|██▌       | 4/16 [00:01<00:05,  2.35it/s]

	 - Recall@14: 0.046875
	 - MRR@14: 0.013879



# 모델링 8
### batch size = 64
### drop rate = 0.5
### learning rate = 0.01
### epochs = 10
### k = 14

In [91]:
args8 = Args(train_df, val_df, test_df, batch_size=64, hsz=50, drop_rate=0.5, lr=0.01, epochs=10, k=k)

In [92]:
model8 = create_model(args8)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(64, 1, 1326)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      206700    
_________________________________________________________________
dropout_7 (Dropout)          (64, 50)                  0         
_________________________________________________________________
dense_7 (Dense)              (64, 1326)                67626     
Total params: 274,326
Trainable params: 274,326
Non-trainable params: 0
_________________________________________________________________


In [93]:
train_model(model8, args8)

Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.0156, train_loss=6.73]

	 - Recall@14 epoch 1: 0.041360
	 - MRR@14    epoch 1: 0.010528



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.0156, train_loss=6.68]

	 - Recall@14 epoch 2: 0.039522
	 - MRR@14    epoch 2: 0.015161



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0, train_loss=6.18]     

...

	 - Recall@14 epoch 8: 0.061581
	 - MRR@14    epoch 8: 0.023855



Train:   0%|          | 0/163 [00:00<?, ?it/s, accuracy=0.188, train_loss=3.84] 

	 - Recall@14 epoch 9: 0.060662
	 - MRR@14    epoch 9: 0.025469



Evaluation:  57%|█████▋    | 17/30 [00:05<00:04,  3.04it/s]

	 - Recall@14 epoch 10: 0.060662
	 - MRR@14    epoch 10: 0.024917



In [94]:
test_model(model8, args8, test_df)

Evaluation:  25%|██▌       | 4/16 [00:01<00:05,  2.38it/s]

	 - Recall@14: 0.066406
	 - MRR@14: 0.022074



# 모델링 9
### batch size = 16
### drop rate = 0.1
### learning rate = 0.01
### epochs = 10
### k = 14

In [99]:
args9 = Args(train_df, val_df, test_df, batch_size=16, hsz=50, drop_rate=0.1, lr=0.01, epochs=10, k=k)

In [100]:
model9 = create_model(args9)

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(16, 1, 1326)]           0         
_________________________________________________________________
GRU (GRU)                    [(16, 50), (16, 50)]      206700    
_________________________________________________________________
dropout_8 (Dropout)          (16, 50)                  0         
_________________________________________________________________
dense_8 (Dense)              (16, 1326)                67626     
Total params: 274,326
Trainable params: 274,326
Non-trainable params: 0
_________________________________________________________________


In [101]:
train_model(model9, args9)

Train:   0%|          | 0/654 [00:00<?, ?it/s, accuracy=0, train_loss=6.71]     

	 - Recall@14 epoch 1: 0.051316
	 - MRR@14    epoch 1: 0.019217



Train:   0%|          | 0/654 [00:00<?, ?it/s, accuracy=0.0625, train_loss=6.06]

	 - Recall@14 epoch 2: 0.049342
	 - MRR@14    epoch 2: 0.019525



Train:   0%|          | 0/654 [00:00<?, ?it/s, accuracy=0.0625, train_loss=4.51]

...

	 - Recall@14 epoch 8: 0.048684
	 - MRR@14    epoch 8: 0.020490



Train:   0%|          | 0/654 [00:00<?, ?it/s, accuracy=0.375, train_loss=2.34]

	 - Recall@14 epoch 9: 0.049342
	 - MRR@14    epoch 9: 0.017688



Evaluation:  79%|███████▊  | 95/121 [00:10<00:02,  8.70it/s]

	 - Recall@14 epoch 10: 0.045395
	 - MRR@14    epoch 10: 0.019729



In [102]:
test_model(model9, args9, test_df)

Evaluation:  66%|██████▌   | 42/64 [00:05<00:03,  7.13it/s]

	 - Recall@14: 0.047619
	 - MRR@14: 0.018790



# 모델링 10
### batch size = 32
### drop rate = 0.5
### learning rate = 0.01
### epochs = 10
### k = 14

In [103]:
args10 = Args(train_df, val_df, test_df, batch_size=32, hsz=50, drop_rate=0.5, lr=0.01, epochs=10, k=k)

In [104]:
model10 = create_model(args10)

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(32, 1, 1326)]           0         
_________________________________________________________________
GRU (GRU)                    [(32, 50), (32, 50)]      206700    
_________________________________________________________________
dropout_9 (Dropout)          (32, 50)                  0         
_________________________________________________________________
dense_9 (Dense)              (32, 1326)                67626     
Total params: 274,326
Trainable params: 274,326
Non-trainable params: 0
_________________________________________________________________


In [105]:
train_model(model10, args10)

Train:   0%|          | 0/327 [00:00<?, ?it/s, accuracy=0, train_loss=7.05]     

	 - Recall@14 epoch 1: 0.037946
	 - MRR@14    epoch 1: 0.013532



Train:   0%|          | 0/327 [00:00<?, ?it/s, accuracy=0.0312, train_loss=6.83]

	 - Recall@14 epoch 2: 0.040179
	 - MRR@14    epoch 2: 0.016909



Train:   0%|          | 0/327 [00:00<?, ?it/s, accuracy=0.0625, train_loss=6.09]

...

	 - Recall@14 epoch 8: 0.055804
	 - MRR@14    epoch 8: 0.023265



Train:   0%|          | 0/327 [00:00<?, ?it/s, accuracy=0.156, train_loss=4.09] 

	 - Recall@14 epoch 9: 0.058036
	 - MRR@14    epoch 9: 0.023671



Evaluation:  70%|███████   | 42/60 [00:07<00:03,  5.37it/s]

	 - Recall@14 epoch 10: 0.055060
	 - MRR@14    epoch 10: 0.023604



In [106]:
test_model(model10, args10, test_df)

Evaluation:  47%|████▋     | 15/32 [00:03<00:03,  4.27it/s]

	 - Recall@14: 0.068750
	 - MRR@14: 0.017053



# 10가지의 모델을 시도한 결과 가장 우수한 모델은 8번 모델이었다.
# model8
### batch size = 64
### drop rate = 0.5
### learning rate = 0.01
### epochs = 10
### k = 14
# Recall@14 : 0.066406
# MRR@14 :0.022074